基于


1.去掉变化最小的特征
说明：如果95%的实例的该特征的取值都是1，那就认为这个特征作用不大，如果100%都是1，该特征没意义。
应用条件：当特征是离散型变量时才可以使用该方法，对于连续型需要先离散化
优缺点：实际上不太会有95%以上都取某个值的特征存在，因此该方法简单但不好用，可以作为特征选择的预处理，先去掉那些取值变化小的特征，然后用其他特征选择方法进行进一步的特征选择。
例：去除有80%以上都是0或都是1的特征

In [3]:
from sklearn.feature_selection import VarianceThreshold
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

2.单变量特征选择
单变量特征选择能够对每一个特征进行测试，衡之间量该特征和响应变量之间的关系，根据得分去除特征。
2.1Pearson相关系数
该方法衡量的是变量之间的线性相关性，结果的取值范围为[-1,1],-1表示完全的负相关，+1表示完全的正相关，0表示没有线性相关。scipy的pearsonr方法能够计算相关系数和p-value

In [7]:
import numpy as np
from scipy.stats import  pearsonr

np.random.seed(0)
size=300
x=np.random.normal(0,1,size)
print("Lower noise",pearsonr(x,x+np.random.normal(0,1,size)))
print("Higher nois",pearsonr(x,x+np.random.normal(0,10,size)))

Lower noise (0.71824836862138408, 7.3240173129983507e-49)
Higher nois (0.057964292079338155, 0.31700993885324752)


说明：该例中比较了变量加入噪声前后的差异，当变量较小时，相关性强。
sklearn提供的f_regression方法能够批量计算特征的p-value。（sklearn的pipeline）
pearson相关系数的缺点：作为特征排序机制，只对线性关系敏感，如果关系是非线性的，即使两个变量具有一一对应的关系，pearson相关性也可能接近0.

注：如果仅仅根据相关系数这个值来判断的话，有时可能有很，最好将数据可视化进行检验。

2.2互信息和最大信息系数
用互信息直接进行特征选择的不便之处：1.它不属于度量方式，也没有办法归一化，在不同的数据集上的结果无法做比较；2.对于连续变量的计算不是很方便（X，y都是集合，都是离散的取值），通常变量需要先离散化，而互信息的结果对离散化的方式很敏感。
最大信息系数克服了这两个问题，它首先寻找一种最优的离散化方法，然后把互信息取值转换成一种度量方式，取值在[0,1]上。

from minepy import MINE

m = MINE()
x = np.random.uniform(-1, 1, 10000)
m.compute_score(x, x**2)
print( m.mic())
MIC的统计能力遭到了一些质疑，当零假设不成立时，MIC的统计就会受到影响。
在有的数据集上不存在这个问题，但有的数据集上就存在这个问题。


2.3 距离相关系数 (Distance correlation)
距离相关系数是为了克服Pearson相关系数的弱点而生的。在x和x^2这个例子中，即便Pearson相关系数是0，我们也不能断定这两个变量是独立的（有可能是非线性相关）；但如果距离相关系数是0，那么我们就可以说这两个变量是独立的。
R的energy包里提供了距离相关系数的实现，另外这是Python gist的实现。
#R-code
> x = runif (1000, -1, 1)
> dcor(x, x**2)
[1] 0.4943864

尽管有MIC和距离相关系数在了，但当变量之间的关系接近线性相关的时候，Pearson相关系数仍然是不可替代的。第一、Pearson相关系数计算速度快，这在处理大规模数据的时候很重要。第二、Pearson相关系数的取值区间是[-1，1]，而MIC和距离相关系数都是[0，1]。这个特点使得Pearson相关系数能够表征更丰富的关系，符号表示关系的正负，绝对值能够表示强度。当然，Pearson相关性有效的前提是两个变量的变化关系是单调的。

2.4 基于学习模型的特征排序 (Model based ranking)
这种方法的思路是直接使用你要用的机器学习算法，针对每个单独的特征和响应变量建立预测模型。其实Pearson相关系数等价于线性回归里的标准化回归系数。假如某个特征和响应变量之间的关系是非线性的，可以用基于树的方法（决策树、随机森林）、或者扩展的线性模型等。基于树的方法比较易于使用，因为他们对非线性关系的建模比较好，并且不需要太多的调试。但要注意过拟合问题，因此树的深度最好不要太大，再就是运用交叉验证。


In [ ]:
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor

#Load boston housing dataset as an example
boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

rf = RandomForestRegressor(n_estimators=20, max_depth=4)
scores = []
for i in range(X.shape[1]):
     score = cross_val_score(rf, X[:, i:i+1], Y, scoring="r2",
                              cv=ShuffleSplit(len(X), 3, .3))
     scores.append((round(np.mean(score), 3), names[i]))
print (sorted(scores, reverse=True))

[(0.68500000000000005, 'LSTAT'), (0.49399999999999999, 'RM'), (0.39900000000000002, 'NOX'), (0.32900000000000001, 'TAX'), (0.28799999999999998, 'PTRATIO'), (0.28100000000000003, 'INDUS'), (0.23000000000000001, 'CRIM'), (0.223, 'RAD'), (0.182, 'ZN'), (0.156, 'B'), (0.104, 'AGE'), (0.070999999999999994, 'DIS'), (-0.0070000000000000001, 'CHAS')]
